In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os
from scipy.spatial import procrustes
data_path = '/Users/jamesburgess/research/robust-dim-reduction-popgen'

In [ ]:
popres_samples = pd.read_csv('{}/data/labels.csv'.format(data_path))
popres_samples.groupby('label').count()